In [ ]:
import sys
import copy
import torch
import numpy as np
import scipy as sp
import pandas as pd
import dill as pickle
import matplotlib.pyplot as plt

In [ ]:
sys.path.append("/home/zanardi/Codes/ML/ROMAr/romar/")
from romar import env
from romar import const
from romar import utils
from romar import backend as bkd
from romar.systems import BoxAd
from romar.roms import CoBRAS

import romar.postproc.plotting as pltt

In [ ]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

In [ ]:
path_to_dtb = "/home/zanardi/Codes/ML/ROMAr/romar/examples/database"
path_to_basis = "/home/zanardi/Codes/ML/ROMAr/run_old2/rad_on_test10/max_mom_2a/cobras_basis.p"
path_to_rom_data = "/home/zanardi/Codes/ML/ROMAr/romar/examples/database/utils/rom_data/"
cases = ["lrho_lT", "lrho_hT", "hrho_lT", "hrho_hT", "mrho_mT"]

In [ ]:
r = 10
use_proj = False

In [ ]:
system = BoxAd(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  rad_dtb=path_to_dtb + "/rates/rad_fit.p",
  use_rad=True,
  use_proj=use_proj,
  use_tables=False
)
system.compute_c_mat(max_mom=2, state_specs=False, include_temp=False)

Plot basis

In [ ]:
basis = pickle.load(open(path_to_basis, "rb"))

In [ ]:
system.rom.build(
  phi=basis["phi"][r],
  psi=basis["psi"][r],
  **{k: basis[k] for k in ("mask", "xref", "xscale")}
)

In [ ]:
x = np.loadtxt(path_to_rom_data + "/x.txt")
zenc_plato = np.loadtxt(path_to_rom_data + "/zenc.txt")
xdec_plato = np.loadtxt(path_to_rom_data + "/xdec.txt")
encoder_plato = np.loadtxt(path_to_rom_data + "/encoder.txt")
decoder_plato = np.loadtxt(path_to_rom_data + "/decoder.txt")

In [ ]:
err = np.abs(system.rom.decoder-decoder_plato) / (np.abs(system.rom.decoder)+1e-35)
err.max()

In [ ]:
err = np.abs(system.rom.encoder-encoder_plato) / (np.abs(system.rom.encoder)+1e-35)
err.max()

In [ ]:
zenc = system.rom.encode(x)
err = np.abs(zenc-zenc_plato) / (np.abs(zenc)+1e-35)
err.max()

In [ ]:
xdec = system.rom.decode(zenc)
err = np.abs(xdec-xdec_plato) / (np.abs(xdec)+1e-35)
err.max()